# Worksheet 08

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [21]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.183195982580061, 4.792384572756389, 5.743165635319489, 3.399533967013456, 4.700078521556765, 4.8054328090445, 5.170314605872873, 4.463885248699418, 5.413764169386793, 4.634202908070582]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [22]:
c2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(c2)

[9.103810951923226, 8.818496980339233, 8.889862115759236, 6.551743650780935, 8.264312847168053, 7.809302744666713, 8.210837952633753, 7.386455667253012, 7.259879320533689, 6.843052495967454]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [23]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[4.634202908070582, 6.843052495967454, 7.259879320533689, 7.386455667253012, 8.210837952633753, 7.809302744666713, 5.413764169386793, 8.264312847168053, 6.551743650780935, 8.889862115759236]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

prior probability 
mean 
variance


d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [24]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2)]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[0])**2, c2)) / len(c2)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.843052495967454, 7.259879320533689, 7.386455667253012, 8.210837952633753, 7.809302744666713, 8.264312847168053, 6.551743650780935, 8.889862115759236]
[4.634202908070582, 5.413764169386793]
P(C_1) = 0.8,  P(C_2) = 0.2
mean_1 = 7.651930849345356,  mean_2 = 5.023983538728688
var_1 = 0.5416882663955696,  var_2 = 7.058036007413609


C:\Users\cxysz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


close to the true value

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [25]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(prob_c[0] * pdf_i[0] / prob_x)
    prob_c1_x.append(prob_c[1] * pdf_i[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  4.634202908070582
probability of observing that point if it came from cluster 0 =  1.3422939731502967e-07
probability of observing that point if it came from cluster 1 =  0.056437002379720484
point =  6.843052495967454
probability of observing that point if it came from cluster 0 =  0.24152598962035152
probability of observing that point if it came from cluster 1 =  0.05467669096291771
point =  7.259879320533689
probability of observing that point if it came from cluster 0 =  0.5667782332570666
probability of observing that point if it came from cluster 1 =  0.05375694347521338
point =  7.386455667253012
probability of observing that point if it came from cluster 0 =  0.6531377008392818
probability of observing that point if it came from cluster 1 =  0.05344381312960524
point =  8.210837952633753
probability of observing that point if it came from cluster 0 =  0.4325027871794663
probability of observing that point if it came from cluster 1 =  0.05104533320301828
point =  7.809

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [26]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum([x * (data[i] - mean[0])**2 for i, x in enumerate(prob_c0_x)]) / sum(prob_c0_x), 
       sum([x * (data[i] - mean[1])**2 for i, x in enumerate(prob_c1_x)]) / sum(prob_c1_x) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.7522778896671696,  P(C_2) = 0.24772211033283043
mean_1 = 7.640074443186303,  mean_2 = 5.5662464135685665
var_1 = 0.5164524736675657,  var_2 = 1.5288316250080012


the updated values of mean_j and var_j should be closer to the true values

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [27]:
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(prob_c[0] * pdf_i[0] / prob_x)
    prob_c1_x.append(prob_c[1] * pdf_i[1] / prob_x)

point =  4.634202908070582
probability of observing that point if it came from cluster 0 =  3.404107776292372e-08
probability of observing that point if it came from cluster 1 =  0.21669262002643835
point =  6.843052495967454
probability of observing that point if it came from cluster 0 =  0.23480574431832066
probability of observing that point if it came from cluster 1 =  0.184117455761509
point =  7.259879320533689
probability of observing that point if it came from cluster 0 =  0.5891130130897232
probability of observing that point if it came from cluster 1 =  0.1412748705811772
point =  7.386455667253012
probability of observing that point if it came from cluster 0 =  0.6847197589784786
probability of observing that point if it came from cluster 1 =  0.12845292998811333
point =  8.210837952633753
probability of observing that point if it came from cluster 0 =  0.4194303710998776
probability of observing that point if it came from cluster 1 =  0.058450946869895874
point =  7.8093027

 the model is more confident in the second set of output, as the probabilities for each point are more heavily skewed towards one cluster.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [28]:
assignments = np.argmax(prob_c0_x)
print(assignments)

5


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      disgree      |
| A  C |      agree      |
| A  D |      agree      |
| A  E |      agree      |
| B  C |      agree      |
| B  D |      agree       |
| B  E |      agree       |
| C  D |      agree       |
| C  E |      agree       |
| D  E |      disagree     |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

N(N-1)/2

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

26

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

for cluster1: 1

for cluster2: 6

total agreement: 3+1+6=10

disagreement: 26

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.


1.Cluster the data using some clustering algorithm to obtain cluster assignments for each data point.

2.For each cluster, create a sub-dataset containing only the points in that cluster.

3.Compute the disagreement distance for each pair of sub-datasets. This can be done by calculating the number of disagreements between the two sub-datasets using the method described in parts (a) and (b) of the previous questions.

4.Combine the disagreement distances for each pair of sub-datasets to obtain a single overall disagreement distance for the entire dataset.1.